In [2]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import random
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/winequality-red.csv')
train = pd.read_csv('./data/winequality-red.csv')

In [3]:
def my_seed_everywhere(seed: int = 42):
    random.seed(seed) # random
    np.random.seed(seed) # np
    os.environ["PYTHONHASHSEED"] = str(seed) # os
    tf.random.set_seed(seed) # tensorflow

my_seed = 42
my_seed_everywhere(my_seed)

In [4]:
train

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [5]:
train['quality'].value_counts()

5    681
6    638
7    199
4     53
8     18
3     10
Name: quality, dtype: int64

In [6]:
y_train = train['quality']

In [7]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
Mytrain_scaled = scaler.fit_transform(train[train.columns[:-1]])

In [8]:
Mytrain_scaled

array([[0.24778761, 0.39726027, 0.        , ..., 0.60629921, 0.13772455,
        0.15384615],
       [0.28318584, 0.52054795, 0.        , ..., 0.36220472, 0.20958084,
        0.21538462],
       [0.28318584, 0.43835616, 0.04      , ..., 0.40944882, 0.19161677,
        0.21538462],
       ...,
       [0.15044248, 0.26712329, 0.13      , ..., 0.53543307, 0.25149701,
        0.4       ],
       [0.11504425, 0.35958904, 0.12      , ..., 0.65354331, 0.22754491,
        0.27692308],
       [0.12389381, 0.13013699, 0.47      , ..., 0.51181102, 0.19760479,
        0.4       ]])

In [15]:
def build_model():
    
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(300, activation='relu', input_shape=(Mytrain_scaled.shape[1],)),
        tf.keras.layers.Dense(300, activation='relu'),
        tf.keras.layers.Dense(300, activation='relu'),
        tf.keras.layers.Dense(1, activation='linear')
  ])

    model.compile(optimizer='Adam',loss='mse',metrics=['mae'])
    
    return model

In [16]:
k = 10
num_val = len(Mytrain_scaled) // k
num_epochs = 100
all_scores = []

for i in range(k):
    print('processing fold #', i)

    X_val = Mytrain_scaled[i * num_val: (i + 1) * num_val]
    y_val = y_train[i * num_val: (i + 1) * num_val]

    X_train_part = np.concatenate([Mytrain_scaled[:i * num_val], Mytrain_scaled[(i + 1) * num_val:]], axis=0)
    y_train_part = np.concatenate([y_train[:i * num_val], y_train[(i + 1) * num_val:]], axis=0)

    model = build_model()
    model.fit(X_train_part, y_train_part, epochs=num_epochs, verbose=1)

    val_loss, val_acc = model.evaluate(X_val, np.array(y_val), verbose=0)
    all_scores.append(val_acc)


processing fold # 0
Epoch 1/100
45/45 [==============================] - 1s 4ms/step - loss: 5.5109 - mae: 1.7891
Epoch 2/100
45/45 [==============================] - 0s 3ms/step - loss: 1.6523 - mae: 1.0148
Epoch 3/100
45/45 [==============================] - 0s 3ms/step - loss: 1.2334 - mae: 0.8742
Epoch 4/100
45/45 [==============================] - 0s 3ms/step - loss: 0.9514 - mae: 0.7598
Epoch 5/100
45/45 [==============================] - 0s 3ms/step - loss: 0.7158 - mae: 0.6648
Epoch 6/100
45/45 [==============================] - 0s 3ms/step - loss: 0.5900 - mae: 0.5996
Epoch 7/100
45/45 [==============================] - 0s 2ms/step - loss: 0.5818 - mae: 0.6036
Epoch 8/100
45/45 [==============================] - 0s 3ms/step - loss: 0.4382 - mae: 0.5201
Epoch 9/100
45/45 [==============================] - 0s 2ms/step - loss: 0.4485 - mae: 0.5242
Epoch 10/100
45/45 [==============================] - 0s 2ms/step - loss: 0.4498 - mae: 0.5284
Epoch 11/100
45/45 [===================

In [38]:
all_scores


[0.5466033220291138,
 0.5164121389389038,
 0.6350245475769043,
 0.5716337561607361,
 0.44048285484313965,
 0.5672088861465454,
 0.5531187653541565,
 0.5277623534202576,
 0.4932571053504944,
 0.5723979473114014]

In [39]:
np.mean(all_scores)

0.5423901677131653

### 결과 1

In [34]:
all_scores


# def build_model():
#   model = tf.keras.Sequential([
#       tf.keras.layers.Dense(256, activation='relu', input_shape=(Mytrain_scaled.shape[1],)),
#       tf.keras.layers.Dense(256, activation='relu'),
#       tf.keras.layers.Dense(256, activation='relu'),
#       tf.keras.layers.Dense(1, activation='linear')
#   ])

#   model.compile(optimizer='Adam',loss='mse',metrics=['mae'])
#   return model

[0.5352317094802856,
 0.5110942125320435,
 0.6228957176208496,
 0.6057754158973694,
 0.44095852971076965,
 0.5820302367210388,
 0.548805832862854,
 0.5074407458305359,
 0.4951742887496948,
 0.5550182461738586]

In [35]:
np.mean(all_scores)

0.54044249355793

### 결과 2

In [41]:
all_scores

# def build_model():
#   model = tf.keras.Sequential([
#       tf.keras.layers.Dense(256, activation='relu', input_shape=(Mytrain_scaled.shape[1],)),
#       tf.keras.layers.Dense(256, activation='relu'),
#       tf.keras.layers.Dense(256, activation='relu'),
#       tf.keras.layers.Dense(256, activation='relu'),
#       tf.keras.layers.Dense(1, activation='linear')
#   ])

#   model.compile(optimizer='Adam',loss='mse',metrics=['mae'])
#   return model

[0.5466033220291138,
 0.5164121389389038,
 0.6350245475769043,
 0.5716337561607361,
 0.44048285484313965,
 0.5672088861465454,
 0.5531187653541565,
 0.5277623534202576,
 0.4932571053504944,
 0.5723979473114014]

In [42]:
np.mean(all_scores)

0.5423901677131653

### 결과 3

In [11]:
# def build_model():
    
#     model = tf.keras.Sequential([
#         tf.keras.layers.Dense(300, activation='relu', input_shape=(Mytrain_scaled.shape[1],)),
#         tf.keras.layers.Dense(300, activation='relu'),
#         tf.keras.layers.Dense(300, activation='relu'),
#         tf.keras.layers.Dense(1, activation='linear')
#   ])

#     model.compile(optimizer='Adam',loss='mse',metrics=['mae'])
    
#     return model
all_scores


[0.5610343813896179,
 0.513828694820404,
 0.6156482696533203,
 0.5560480356216431,
 0.409481406211853,
 0.5750181078910828,
 0.5663350224494934,
 0.5484398603439331,
 0.47413933277130127,
 0.5871073007583618]

In [12]:
np.mean(all_scores)

0.540708041191101

### 결과 4

In [17]:
# def build_model():
    
#     model = tf.keras.Sequential([
#         tf.keras.layers.BatchNormalization(),
#         tf.keras.layers.Dense(300, activation='relu', input_shape=(Mytrain_scaled.shape[1],)),
#         tf.keras.layers.Dense(300, activation='relu'),
#         tf.keras.layers.Dense(300, activation='relu'),
#         tf.keras.layers.Dense(1, activation='linear')
#   ])

#     model.compile(optimizer='Adam',loss='mse',metrics=['mae'])
    
#     return model
all_scores

[0.5258644223213196,
 0.5361930131912231,
 0.6263356804847717,
 0.6622455716133118,
 0.4513867497444153,
 0.5916872620582581,
 0.6034874320030212,
 0.5757561922073364,
 0.5588325262069702,
 0.6137396097183228]

In [18]:
np.mean(all_scores)

0.574552845954895

### 결론

### 결과 1이 0.54044249355793 로 제일 높음